# 01 Multi-Factor Regression with Controls (Micro)

Add controls, discuss omitted variable bias, robust SE.


## Table of Contents
- Choose controls
- Fit model
- Compare coefficients


## Why This Notebook Matters
Regression is the bridge between statistics and ML. You will learn:
- single-factor vs multi-factor interpretation,
- robust standard errors,
- coefficient stability and multicollinearity.


## What You Will Produce
- (no file output; learning/analysis notebook)

## Success Criteria
- You can explain what you built and why each step exists.
- You can run your work end-to-end without undefined variables.

## Common Pitfalls
- Running cells top-to-bottom without reading the instructions.
- Leaving `...` placeholders in code cells.
- Treating coefficients as causal without a causal design.
- Ignoring multicollinearity (unstable coefficients).

## Matching Guide
- `docs/guides/02_regression/01_multifactor_regression_micro_controls.md`



## How To Use This Notebook
- This notebook is hands-on. Most code cells are incomplete on purpose.
- Complete each TODO, then run the cell.
- Use the matching guide (`docs/guides/02_regression/01_multifactor_regression_micro_controls.md`) for deep explanations and alternative examples.
- Write short interpretation notes as you go (what changed, why it matters).



## Environment Bootstrap
Run this cell first. It makes the repo importable and defines common directories.


In [ ]:
from __future__ import annotations

from pathlib import Path
import sys


def find_repo_root(start: Path) -> Path:
    p = start
    for _ in range(8):
        if (p / 'src').exists() and (p / 'docs').exists():
            return p
        p = p.parent
    raise RuntimeError('Could not find repo root. Start Jupyter from the repo root.')


PROJECT_ROOT = find_repo_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
SAMPLE_DIR = DATA_DIR / 'sample'

PROJECT_ROOT



## Goal
Fit a multi-factor regression with controls and discuss omitted variable bias.


## Your Turn: Choose controls


In [ ]:
# TODO: Choose at least 2 controls (example: log(population), poverty_rate)
# Compare coefficient on log(income) with vs without controls
...


## Your Turn: Robust SE + Optional Clustering


In [ ]:
# TODO: Fit HC3 robust SE
# Advanced: cluster by state (research statsmodels cov_type='cluster')
...


## Checkpoint (Self-Check)
Run a few asserts and write 2-3 sentences summarizing what you verified.



In [ ]:
# TODO: After you build X/y and split by time, validate the split.
# Example (adjust variable names):
# assert X_train.index.max() < X_test.index.min()
# assert y_train.index.equals(X_train.index)
# assert y_test.index.equals(X_test.index)
# assert not X_train.isna().any().any()
# assert not X_test.isna().any().any()
...



## Extensions (Optional)
- Try one additional variant beyond the main path (different features, different split, different model).
- Write down what improved, what got worse, and your hypothesis for why.



## Reflection
- What did you assume implicitly (about timing, availability, stationarity, or costs)?
- If you had to ship this model, what would you monitor?



## Solutions (Reference)

Try the TODOs first. Use these only to unblock yourself or to compare approaches.

<details><summary>Solution: Choose controls</summary>

```python
import numpy as np
import pandas as pd
from src import econometrics

df = pd.read_csv(SAMPLE_DIR / 'census_county_sample.csv')
df = df[(df['B19013_001E'] > 0) & (df['B25064_001E'] > 0) & (df['B01003_001E'] > 0)].copy()
df['log_income'] = np.log(df['B19013_001E'].astype(float))
df['log_rent'] = np.log(df['B25064_001E'].astype(float))
df['log_pop'] = np.log(df['B01003_001E'].astype(float))

res = econometrics.fit_ols_hc3(df, y_col='log_rent', x_cols=['log_income', 'log_pop', 'poverty_rate'])
print(res.summary())
```

</details>

<details><summary>Solution: Fit model</summary>

```python
# See above.
```

</details>

<details><summary>Solution: Compare coefficients</summary>

```python
# Compare single-factor vs multi-factor slopes for log_income.
# If it changes a lot, omitted variable bias is plausible.
```

</details>

